# 태스크 3: 질문 답변을 위한 Amazon Bedrock 사용

이 노트북에서는 Bedrock Titan 모델을 사용하여 전체 관련 컨텍스트가 포함된 요청을 모델에 보내고 응답이 돌아올 것으로 예상하여 정보 응답을 쿼리에 제공함으로써 문서를 미리 준비하고 색인을 생성할 필요 없이 질문에 대해 모델에서 사실적인 답변을 반환하도록 하는 과제를 해결하는 방법을 알아봅니다.

이 노트북에서는 **검색 증강 생성(RAG)**의 역할을 시뮬레이션하지만, 실제로 RAG를 사용하지는 않습니다. 이 접근 방식은 짧은 문서 또는 단독 애플리케이션에 작동하며, 모델에 전송된 프롬프트에 완전히 맞지는 않을 수 있는 대규모 기업 문서가 있는 경우 기업 수준의 질문 답변으로 규모를 조정하지 못할 수 있습니다.

**질문 답변(QA)**은 자연어로 제기되는 사실적인 쿼리에 대한 답변을 추출하는 것이 포함된 중요한 태스크입니다. 일반적으로 QA 시스템은 정형 또는 비정형 데이터가 포함된 기술 자료에 대해 쿼리를 처리하고, 정확한 정보로 응답을 생성합니다. 높은 정확도를 보장하는 것은 특히 엔터프라이즈 사용 사례를 위한 유용하고 안정적이고 신뢰할 수 있는 질의응답 시스템을 개발하는 데 있어서 핵심적인 요인입니다.


## 시나리오

AnyCompany에서 제조하는 특정 차량 모델의 타이어 교체에 대한 정보를 제공하기 위해 질문에 답변하는 모델을 묻는 상황을 모델링해 봅니다. 먼저 "제로샷" 접근 방식으로 모델을 쿼리하여 훈련 데이터만을 기반으로 관련 답변을 제공할 수 있는지 확인합니다.

그런데 가짜 차량 모델을 시험해 보고 비슷한 응답을 받았을 때 드러났듯이 이 모델에서는 더 일반적인 답변을 "환각"하는 듯하다는 것을 깨닫습니다. 이는 각 모델의 타이어에 대한 세부 사항을 제공하는 Example Company의 실제 차량 설명서로 모델의 훈련을 보강할 필요가 있다는 것을 암시합니다.

이 실습에서는 외부 데이터 없이 이러한 "검색 증강 생성"(RAG) 접근 방식을 시뮬레이션합니다. AnyCompany 모델 Z 차량의 타이어 교체 방법을 설명하는 자세한 설명서 발췌를 제공합니다. 이제 모델에서 상황에 맞는 예시 콘텐츠를 활용하여 사용자 지정된 정확한 답변을 제공할 수 있는지 테스트합니다.

## 태스크 3.1: 환경 설정

이 태스크에서는 환경을 설정합니다.

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## 태스크 3.2: 모델 지식을 사용한 Q&amp;A
이 섹션에서는 Bedrock 서비스에서 제공한 모델을 사용하여 훈련 단계 동안 획득한 지식을 토대로 질문에 답변하려고 합니다.

이 태스크에서는 Amazon Bedrock 클라이언트의 invoke_model() 메서드를 사용합니다. 이 메서드를 사용하는 데 필요한 필수 파라미터는 Amazon Bedrock 모델 ARN을 나타내는 modelId와 태스크에 대한 프롬프트인 body입니다.

body 프롬프트는 선택하는 파운데이션 모델 공급자에 따라 변경됩니다. 아래에서 이 작업을 자세히 연습해 봅니다.

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

Bedrock 서비스에서 제공한 모델을 사용하여 훈련 단계 동안 획득한 지식을 토대로 질문에 답변하려고 합니다.

In [ ]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my AnyCompany AC8?
Answer:"""
parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }

## 태스크 3.3: JSON 본문을 전달하여 모델을 호출하고 응답 생성

In [ ]:
#model configuration
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"
try:
    
    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body.get("results")[0].get("outputText")
    print(answer.strip())

except botocore.exceptions.ClientError as error:
    if  error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error


이 모델은 바람이 빠진 자동차 타이어를 교체하는 프로세스를 설명하는 답변을 제공하지만, 동일한 설명이 모든 자동차에 유효할 수 있습니다. 안타깝게도 예비 타이어가 없는 AnyCompany AC8에는 올바른 답변이 아닙니다. 모델이 자동차의 타이어 교체에 대한 지침이 포함된 데이터로 훈련되었기 때문입니다.

가상의 자동차 브랜드와 모델인 Amazon Tirana에 대해 동일한 질문을 제시하면 이 문제의 또 다른 예를 확인할 수 있습니다.

In [ ]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
body = json.dumps({"inputText": prompt_data, 
                   "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

프롬프트 질문이 주어지면 모델에서 현실적인 답변을 제공할 수 없습니다.

이 문제를 해결하고 모델에서 차량 모델에 유효한 특정 지침을 기반으로 답변을 제공하도록 하려면 프롬프트의 일부로 추가 지식 기반을 제공하여 모델의 지식을 즉시 보강할 수 있습니다.

이 방법을 사용하여 애플리케이션을 개선하는 방법을 살펴보겠습니다.

다음은 AnyCompany AC8 설명서에서 발췌한 것입니다(사실 실제 설명서는 아니지만 그렇게 취급해 주십시오). 이 문서도 편의를 위해 Titan Large 컨텍스트 창에 완전히 맞을 수 있게 충분히 짧게 작성되었습니다.

```plain
타이어 및 타이어 압력:

타이어는 검은색 고무로 만들어져 있으며 차량 바퀴에 탑재됩니다. 주행, 코너링 및 제동에 필요한 그립을 제공합니다. 고려해야 할 두 가지 중요한 요인은 타이어 압력과 타이어 마모입니다. 자동차의 성능과 핸들링에 영향을 줄 수 있기 때문입니다.

권장되는 타이어 압력을 확인할 수 있는 위치:

운전자의 측면 B 필라에 있는 압력 레이블에서 권장되는 압력 사양을 찾을 수 있습니다. 또는 차량 설명서에서 이 정보를 참조할 수 있습니다. 권장되는 타이어 압력은 속도, 탑승자 수 또는 차량의 최대 하중에 따라 달라질 수 있습니다.

타이어에 다시 공기 주입:

압력은 타이어가 차가울 때 확인하는 것이 중요합니다. 즉, 차량을 3시간 이상 정차하여 타이어가 주변 온도와 같은 온도가 되도록 할 수 있습니다.

타이어에 다시 공기를 주입하려면:

    차량의 권장 타이어 압력을 확인하십시오.
    에어 펌프에 대해 제공된 지침을 따르고 올바른 압력이 될 때까지 타이어에 공기를 주입합니다.
    차량의 중앙 디스플레이에서 "Car status"(자동차 상태) 앱을 엽니다.
    "Tire pressure"(타이어 압력) 탭으로 이동합니다.
    "Calibrate pressure"(압력 보정) 옵션을 누르고 작업을 확인합니다.
    자동차를 몇 분 동안 30km/h 이상 속도로 구동하여 타이어 압력을 보정합니다.

참고: 일부 경우에 타이어 압력과 관련된 경고 기호 또는 메시지를 지우기 위해 15분 넘게 운전해야 할 수 있습니다. 경고가 지속되면 타이어를 냉각하고 위 단계를 반복하십시오.

타이어 바람이 빠짐:

구동하는 동안 타이어에서 바람이 빠질 경우 펑크난 부분을 임시로 봉인하고, 타이어 이동 키트를 사용하여 타이어에 다시 바람을 넣을 수 있습니다. 이 키트는 일반적으로 차량의 짐칸 덮개 아래에 보관됩니다.

타이어 수리 키트 사용 지침:

    차량의 뒷문이나 트렁크를 엽니다.
    짐칸 덮개를 들어 올려 타이어 수리 키트에 접근합니다.
    타이어 수리 키트에 제공된 지침에 따라 타이어 펑크 부분을 때웁니다.
    키트를 사용한 후에는 원래 위치에 안전하게 다시 보관해야 합니다.
    사용한 실란트병의 폐기 및 교체에 대해 도움이 필요한 경우 Rivesla 또는 해당 서비스 팀에 문의하십시오.

타이어 수리 키트는 임시 해결책이며, 최대 속도 시속 80km에서 최대 10분 또는 8km(더 빠른 경우)까지 주행할 수 있도록 설계되었습니다. 펑크난 타이어를 교체하거나 가능한 한 빨리 전문가의 수리를 받는 것이 좋습니다.
```

In [ ]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact AnyCompany or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

##### 이제 전체 발췌문을 질문과 함께 모델에 전달합니다.

In [ ]:
question = "How can I fix a flat tire on my AnyCompany AC8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

### 태스크 3.4: boto3를 통해 모델을 호출하여 응답 생성

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

모델에서 컨텍스트를 이해하고 관련 답변을 생성하는 데 시간이 걸리기 때문에 응답을 몇 초 동안 기다려야 하므로 사용자 경험이 저하될 수 있습니다.

또한 Bedrock은 모델에서 토큰을 생성할 때 서비스에서 출력을 생성하는 스트리밍 기능도 지원합니다. 다음은 이를 구현하는 방법의 예입니다.

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
# response with stream
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

이 응답은 타이어 교체 방법에 대한 요약된 단계별 지침을 제공합니다. 

지금까지 검색 증강 생성(RAG) 또는 증강 프로세스를 활용하여 제공된 특정 컨텍스트와 정보에 알맞게 큐레이팅된 응답을 생성할 수 있는 방법을 알아보았습니다.

### 직접 해보기
- 프롬프트를 특정 사용 사례로 변경하고 다양한 모델의 출력을 평가하십시오.
- 토큰 길이를 조작하면서 서비스의 지연 시간 및 응답성을 이해해 보십시오.
- 다양한 프롬프트 엔지니어링 원칙을 적용하여 더 나은 출력을 얻을 수 있습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫고 **태스크 4**를 계속합니다.